## Using MCST to play tic-tac toe

In [9]:
import numpy as np
import random
import math

In [10]:

class tic_tac():

  def __init__(self, globalHash):
    self.state_count = 0
    self.state_hash = globalHash
    self.istate = [[0, 0, 0],
                  [0, 0, 0],
                  [0, 0, 0],]
    # if tuple(tuple(p) for p in self.istate) not in self.state_hash.keys():
    #   self.state_count += 1
    #   self.state_hash[tuple(tuple(p) for p in self.istate)] = self.state_count
  
  def first_state(self):
    return self.istate

  def next_player(self, player):
    if player == 1:
      return 2
    return 1

  def copy_list(self, l1):
    l2 = [ [0]*3 for i in range(3)]
    for i in range (3):
      for j in range (3):
        l2[i][j] = l1[i][j]
    return l2

  def count_empty(self, current_state):
    count = 0
    for row in current_state:
      for element in row:
        if element == 0:
          count += 1
    return count

  def win_check(self,current_state): 
    rowX = [] 
    colX = [] 
    diagX = []
    rowO = []  
    colO = [] 
    diagO = []
    
    for row in current_state:
      rowX.append(row[0] == 'X' and row[1] == 'X' and row[2] == 'X')
      rowO.append(row[0] == 'O' and row[1] == 'O' and row[2] == 'O')
    
    myArray = np.array(current_state).T
    for col in myArray:
      colX.append(col[0] == 'X' and col[1] == 'X' and col[2] == 'X')
      colO.append(col[0] == 'O' and col[1] == 'O' and col[2] == 'O')
    
    diagX.append(current_state[0][0] == 'X' and current_state[1][1] == 'X' and current_state[2][2] == 'X')
    diagX.append(current_state[2][0] == 'X' and current_state[1][1] == 'X' and current_state[0][2] == 'X')

    diagO.append(current_state[0][0] == 'O' and current_state[1][1] == 'O' and current_state[2][2] == 'O')
    diagO.append(current_state[2][0] == 'O' and current_state[1][1] == 'O' and current_state[0][2] == 'O')


    if ((1 in set(rowX)) or (1 in set(colX)) or (1 in set(diagX))):
      return 1
    elif ((1 in set(rowO)) or (1 in set(colO)) or (1 in set(diagO))):
      return 2
    else:
      return 0
 
  def terminal_state(self, current_state):
    count = 0
    if self.win_check(current_state):
      return True
    for row in current_state:
      for element in row:
        if element == 0:
          return False
    return True

  def possible_states(self, current_state, player):   
    if player == 1:
      symbol = 'X'
    else:
      symbol = 'O'

    possible_states = []
    
    if not self.terminal_state(current_state):
      for row in range(3):
        for col in range(3):
          copy_state = self.copy_list(current_state)   #resetting the copy state everytime
          if current_state[row][col] == 0:  
            copy_state[row][col] = symbol           
            #if tuple(tuple(p) for p in copy_state) not in self.state_hash.keys():
              #self.state_count += 1
              #self.state_hash[tuple(tuple(p) for p in copy_state)] = self.state_count
            possible_states.append(copy_state)
   
    return(possible_states)


In [11]:
def printBoard(board):
  print("\n")
  print("_"*39)
  for row in board:
    print("\n")
    for element in row:
      if element != 0:
        print("|    {}    |".format(element), end='')
      else:
         print("|    {}    |".format("_"), end='')
  print("\n")
  print("_"*39)

In [12]:
class treeNode():
  def __init__(self, nowState, parent = None):
    self.used = 0
    self.UCB = 0
    self.simulation = 0
    self.pSimulation = 0
    self.parentNode = parent
    self.win = 0
    self.stateID = tuple(tuple(p) for p in nowState)
    self.state = nowState 
    self.childList = []

In [13]:
class MonteCarloSTree():
  
  
  def __init__ (self, gameInstance, nowPlayer): 
    print("Monte Carlo Agent Created")
    self.nowPlayer = nowPlayer
    self.myGame = gameInstance
    self.hashMap = gameInstance.state_hash
    self.countStates = gameInstance.state_count
    self.treeDict = {}
    self.hashNodeMap = {} 
    self.rootState = gameInstance.first_state()
    self.rootNode = treeNode(self.rootState)
  
  def simulation(self, nowState, player):
    while(not self.myGame.terminal_state(nowState)):
      possibleStates = self.myGame.possible_states(nowState, player)
      nowState = random.choice(possibleStates)

    if self.myGame.win_check(nowState) == player:
      return 5
    elif self.myGame.win_check(nowState) == ((player%2)+1):
        return -100
    else:
      return 4.5
     


  def rollout(self, nowNode, player):
    
    nowState = nowNode.state
    if self.myGame.terminal_state(nowState):
      if self.myGame.win_check(nowState) == player:
        return 5
      elif self.myGame.win_check(nowState) == ((player%2)+1):
        return -100
      else:
        return 4.5
     
    else:
      possibleStates = self.myGame.possible_states(nowState, player)
      nowState = random.choice(possibleStates)
      return self.simulation(nowState, self.myGame.next_player(player)) 
  


  def hashVal(self, state):
    return tuple(tuple(t) for t in state)
  
  
  ## we need a dictionary to map the state hash value with node objects
  ## updated it in the myGame original hash table
  
  def UCBscore(self, node):
    V = node.win/node.simulation
    c = 200000
    if node.simulation == 0 or node.pSimulation == 0:
      UCB = 0
      return UCB
    lTerm = math.sqrt(math.log(node.pSimulation)/node.simulation)
    UCB = V + c*lTerm
    return UCB


  def backProp(self, aNode):
    pointer = aNode
    while(pointer.parentNode != None):
      pointer.parentNode.simulation += aNode.simulation
      pointer.parentNode.win += aNode.win
      pointer.pSimulation = pointer.parentNode.simulation
      pointer.parentNode.UCB = self.UCBscore(pointer.parentNode)
      pointer = pointer.parentNode

  
  def findRoot(self, aNode):
    pointer = aNode
    while(pointer.parentNode != None):
      pointer = pointer.parentNode
    return pointer


  def constructTree(self, nowNode, nowPlayer):
  
   
    hashV = self.hashVal(nowNode.state)
    if hashV  not in self.myGame.state_hash.keys():
      self.myGame.state_hash[hashV] = nowNode 
    
    if len(nowNode.childList) == 0:
      if nowNode.used == 0:  
        nowNode.simulation += 1
        nowNode.win += self.rollout(nowNode, nowPlayer)
        
        nowNode.used = 1
        nowNode.UCB = self.UCBscore(nowNode)
      else:
        nowNode.simulation += 1
        possible_actions = self.myGame.possible_states(nowNode.state, nowPlayer)
        
        if len(possible_actions) != 0:
          for state in possible_actions:
            newNode = treeNode(state, nowNode) ## second variable is the parent node
            newNode.pSimulation = nowNode.simulation
            newNode.parentNode = nowNode
            self.myGame.state_hash[self.hashVal(state)] = newNode
            nowNode.childList.append(newNode)             
          
          nextNode = nowNode.childList[0]         
          nowNode.win += self.rollout(nextNode, self.myGame.next_player(nowPlayer))
        
        else:          
          if self.myGame.win_check(nowNode.state) == nowPlayer:
            nowNode.win += 5
          elif self.myGame.win_check(nowNode.state) == ((nowPlayer%2)+1):
            nowNode.win += -100
          else:
            nowNode.win += 4.5

          
    else:    
      nowNode.simulation += 1
      ucbScoreList = []
      
      for node in nowNode.childList:
        nowNode.win += node.win 
        ucbScoreList.append(node.UCB)
      
      ucbScoreList = np.array(ucbScoreList)
      maxIndex = np.argmax(ucbScoreList)
      if 0 in ucbScoreList:
        index0 = int(np.where(ucbScoreList == 0)[0][0])
        self.constructTree(nowNode.childList[index0], nowPlayer%2+1)
      else:
        self.constructTree(nowNode.childList[maxIndex], nowPlayer%2+1)
    
    
    self.backProp(nowNode)
    return self.findRoot(nowNode)
  
  
  def nextMove(self, state):
    currHash = self.hashVal(state)

    if(currHash not in self.myGame.state_hash.keys()):
      print("Insufficient training, train more,  can't play, exiting")
      return -1
    
    nowNode = self.myGame.state_hash[currHash]
    terminalCheck = self.myGame.terminal_state(state)
    if (terminalCheck):
      print("No more possible moves.")
    else:
      ucbScoreList = []
      
      for node in nowNode.childList:
        ucbScoreList.append(node.UCB)
      print("No. of possible moves from here:", len(ucbScoreList))
      if len(ucbScoreList) == 0:
        print("Train more")
        return -1
      ucbScoreList = np.array(ucbScoreList)
      maxIndex = np.argmax(ucbScoreList)
      bestNode = nowNode.childList[maxIndex]
      print(type(bestNode))
      return bestNode
      

In [14]:
##game = tic_tac()
##mcst = MonteCarloSTree(game)
## n is no. of simulations
def train(mcst, n, player):
  for i in range(n):
    mcst.rootNode = mcst.constructTree(mcst.rootNode, player)
  return mcst

In [15]:
def playOnGo(n):
  hash1 = {}
  aGame = tic_tac(hash1)
  
  currentState = aGame.istate
  turn = 1
  print("Current state of the board:------")
  printBoard(currentState)
  while not aGame.terminal_state(currentState):
    if turn == 1:
      print("Select one of the following states:")
      possibleMoves = aGame.possible_states(currentState, 1)
      index = 0
      for state in possibleMoves:
        
        print("***************Option{}***************\n".format(index))
        printBoard(state)
        index += 1
      option = int(input("Enter your choice here:"))
      currentState = possibleMoves[option]
      printBoard(currentState)
      turn = 2
    else:
      hash1 = {}
      aGame = tic_tac(hash1)
      mcst = MonteCarloSTree(aGame, 2)
      newNode = treeNode(currentState)
      # hashV = mcst.hashVal(newNode.state)
      # if hashV  not in aGame.state_hash.keys():
      mcst.rootNode = newNode
      # else:
      #   mcst.rootNode = aGame.state_hash[hashV]

      print("The computer is thinking..................")
      mcst = train(mcst, n, 2)
      nextNode = mcst.nextMove(currentState)
      currentState = nextNode.state
      print("\nCurrent configuration of the board: \n")
      printBoard(currentState)
      turn = 1

  if aGame.win_check(currentState) == 1:
    print("Player 1 has won!!")
  elif aGame.win_check(currentState) == 2:
    print("Player 2 has won!!")
  else:
    print("ehhh, a draw.")
       

Play Single Player game with the computer. *Change n for no. of simulations.* **key note: the computer playing good game strictly depends upon the no. of simulations. We need very high number of simulations to make it play a perfect game, which takes a long, long time. It is also dependent on the value of constant *c* in the UCB formula**

In [16]:
n = 200000  # no. of simulations. change this value for varying no. of simulations on the monte carlo search tree.
playOnGo(n)

Current state of the board:------


_______________________________________


|    _    ||    _    ||    _    |

|    _    ||    _    ||    _    |

|    _    ||    _    ||    _    |

_______________________________________
Select one of the following states:
***************Option0***************



_______________________________________


|    X    ||    _    ||    _    |

|    _    ||    _    ||    _    |

|    _    ||    _    ||    _    |

_______________________________________
***************Option1***************



_______________________________________


|    _    ||    X    ||    _    |

|    _    ||    _    ||    _    |

|    _    ||    _    ||    _    |

_______________________________________
***************Option2***************



_______________________________________


|    _    ||    _    ||    X    |

|    _    ||    _    ||    _    |

|    _    ||    _    ||    _    |

_______________________________________
***************Option3***************



________________

Testing accuracy when: 

(i) Computer is one move away from win:

In [19]:
sudoState = [['X', 0,  0],
             [ 0 , 0,  0 ],
             ['X','O','O']]

printBoard(sudoState)
print("Computer is playing X, player 1.")
aHash = {}
aGame = tic_tac(aHash)
mcst = MonteCarloSTree(aGame, 1) # 1 becasue it's playing as player 1
newNode = treeNode(sudoState)
mcst.rootNode = newNode
n = 500000 # n is the number of simulations used by the montecarlo agent
print("The computer is calculating it's next move........")
mcst = train(mcst, n, 1)  # PLAYER 1
nextNode = mcst.nextMove(sudoState)
currentState = nextNode.state
print("\nCurrent configuration of the board: \n")
printBoard(currentState)
if aGame.win_check(currentState) == 1:
  print("Player 1 has won!!")
else:
  print("Too bad, it didn't win.")



_______________________________________


|    X    ||    _    ||    _    |

|    _    ||    _    ||    _    |

|    X    ||    O    ||    O    |

_______________________________________
Computer is playing X, player 1.
Monte Carlo Agent Created
The computer is calculating it's next move........
No. of possible moves from here: 5
<class '__main__.treeNode'>

Current configuration of the board: 



_______________________________________


|    X    ||    _    ||    _    |

|    X    ||    _    ||    _    |

|    X    ||    O    ||    O    |

_______________________________________
Player 1 has won!!


(ii) The computer is one move away from loss

As we can see, the tree is not able to predict the correct move here. But with ample amount of simulations, accompanied with chossing correct reward and correct value of constant *c* in UCB calculations, it should converge to correct solution overtime. I've tried with a lot of different values but it is taking too much time. I'm afraid I find myself bounded by the hardware resources I have access to.

In [20]:
sudoState = [[ 0  , 0 , 'X' ],
             [ 0 , 'O' , 0 ],
             ['X' , 0 ,'O']]

printBoard(sudoState)
print("Computer is playing X, player 1.")
aHash = {}
aGame = tic_tac(aHash)
mcst = MonteCarloSTree(aGame, 1) # 1 becasue it's playing as player 1
newNode = treeNode(sudoState)
mcst.rootNode = newNode
n = 500000 # n is the number of simulations used by the montecarlo agent
print("The computer is calculating it's next move........")
mcst = train(mcst, n, 1)  # PLAYER 1
nextNode = mcst.nextMove(sudoState)
currentState = nextNode.state
print("\nCurrent configuration of the board: \n")
printBoard(currentState)




_______________________________________


|    _    ||    _    ||    X    |

|    _    ||    O    ||    _    |

|    X    ||    _    ||    O    |

_______________________________________
Computer is playing X, player 1.
Monte Carlo Agent Created
The computer is calculating it's next move........
No. of possible moves from here: 5
<class '__main__.treeNode'>

Current configuration of the board: 



_______________________________________


|    _    ||    _    ||    X    |

|    X    ||    O    ||    _    |

|    X    ||    _    ||    O    |

_______________________________________


#### playing 1000 games against itself

In [22]:
def playPC(n, nGames = 1):
  hash1 = {}
  aGame = tic_tac(hash1)
  awin = bwin = draw = 0
  
  for i in range (nGames):
    
    currentState = aGame.istate
    print("Current state of the board:------")
    printBoard(currentState)
    turn = 1
    while not aGame.terminal_state(currentState):
      if turn == 1:
        hash1 = {}
        aGame = tic_tac(hash1)
        mcst1 = MonteCarloSTree(aGame, turn)
        newNode = treeNode(currentState)
        # hashV = mcst1.hashVal(newNode.state)
        # if hashV  not in aGame.state_hash.keys():
        mcst1.rootNode = newNode
        # else:
          # mcst1.rootNode = aGame.state_hash[hashV]

        print("The computer{} is thinking..................".format(turn))
        mcst1 = train(mcst1, n, turn)
        nextNode = mcst1.nextMove(currentState)
        currentState = nextNode.state
        print("\nCurrent configuration of the board: \n")
        printBoard(currentState)
        turn = 2
      else:

        hash2 = {}
        bGame = tic_tac(hash2)
        mcst2 = MonteCarloSTree(bGame, turn)
        newNode = treeNode(currentState)
        # # # hashV = mcst2.hashVal(newNode.state)
        # # # if hashV not in bGame.state_hash.keys():
        mcst2.rootNode = newNode
        # # else:
        # #   mcst2.rootNode = bGame.state_hash[hashV]

        print("The computer{} is thinking..................".format(turn))
        mcst2 = train(mcst2, n, turn)
        nextNode = mcst2.nextMove(currentState)
        
        currentState = nextNode.state
        print("\nCurrent configuration of the board: \n")
        printBoard(currentState)
        turn = 1


    if aGame.win_check(currentState) == 1:
      print("Player 1 has won!!")
      awin += 1
    elif aGame.win_check(currentState) == 2:
      bwin += 1
      print("Player 2 has won!!")
    else:
      draw += 1
      print("ehhh, a draw.")
       

  return awin, bwin, draw


In [23]:
nSimulations = 1000 # no of simulations for each game (increasing this will give better results, but take lot of time.)
nGames = 1000 # total no. of games
a, b, c = playPC(nSimulations, nGames)
print("no. of games :{}".format(nGames))
print("no. of games won by player1:{}".format(a))
print("no. of games won by player2:{}".format(b))
print("no. of games drawn:{}".format(c))

Current state of the board:------


_______________________________________


|    _    ||    _    ||    _    |

|    _    ||    _    ||    _    |

|    _    ||    _    ||    _    |

_______________________________________
Monte Carlo Agent Created
The computer1 is thinking..................
No. of possible moves from here: 9
<class '__main__.treeNode'>

Current configuration of the board: 



_______________________________________


|    _    ||    _    ||    _    |

|    _    ||    X    ||    _    |

|    _    ||    _    ||    _    |

_______________________________________
Monte Carlo Agent Created
The computer2 is thinking..................
No. of possible moves from here: 8
<class '__main__.treeNode'>

Current configuration of the board: 



_______________________________________


|    _    ||    _    ||    _    |

|    _    ||    X    ||    _    |

|    _    ||    _    ||    O    |

_______________________________________
Monte Carlo Agent Created
The computer1 is thinking....

No. of possible moves from here: 9
<class '__main__.treeNode'>

Current configuration of the board: 



_______________________________________


|    _    ||    _    ||    _    |

|    _    ||    X    ||    _    |

|    _    ||    _    ||    _    |

_______________________________________
Monte Carlo Agent Created
The computer2 is thinking..................
No. of possible moves from here: 8
<class '__main__.treeNode'>

Current configuration of the board: 



_______________________________________


|    _    ||    _    ||    _    |

|    _    ||    X    ||    O    |

|    _    ||    _    ||    _    |

_______________________________________
Monte Carlo Agent Created
The computer1 is thinking..................
No. of possible moves from here: 7
<class '__main__.treeNode'>

Current configuration of the board: 



_______________________________________


|    _    ||    _    ||    _    |

|    _    ||    X    ||    O    |

|    _    ||    _    ||    X    |

___________________________

No. of possible moves from here: 5
<class '__main__.treeNode'>

Current configuration of the board: 



_______________________________________


|    O    ||    X    ||    _    |

|    O    ||    X    ||    _    |

|    _    ||    X    ||    _    |

_______________________________________
Player 1 has won!!
Current state of the board:------


_______________________________________


|    _    ||    _    ||    _    |

|    _    ||    _    ||    _    |

|    _    ||    _    ||    _    |

_______________________________________
Monte Carlo Agent Created
The computer1 is thinking..................
No. of possible moves from here: 9
<class '__main__.treeNode'>

Current configuration of the board: 



_______________________________________


|    _    ||    _    ||    _    |

|    _    ||    X    ||    _    |

|    _    ||    _    ||    _    |

_______________________________________
Monte Carlo Agent Created
The computer2 is thinking..................
No. of possible moves from here: 8
<

No. of possible moves from here: 7
<class '__main__.treeNode'>

Current configuration of the board: 



_______________________________________


|    _    ||    _    ||    _    |

|    _    ||    _    ||    O    |

|    X    ||    X    ||    _    |

_______________________________________
Monte Carlo Agent Created
The computer2 is thinking..................
No. of possible moves from here: 6
<class '__main__.treeNode'>

Current configuration of the board: 



_______________________________________


|    _    ||    O    ||    _    |

|    _    ||    _    ||    O    |

|    X    ||    X    ||    _    |

_______________________________________
Monte Carlo Agent Created
The computer1 is thinking..................
No. of possible moves from here: 5
<class '__main__.treeNode'>

Current configuration of the board: 



_______________________________________


|    _    ||    O    ||    _    |

|    _    ||    _    ||    O    |

|    X    ||    X    ||    X    |

___________________________

No. of possible moves from here: 5
<class '__main__.treeNode'>

Current configuration of the board: 



_______________________________________


|    _    ||    _    ||    O    |

|    _    ||    X    ||    O    |

|    X    ||    X    ||    _    |

_______________________________________
Monte Carlo Agent Created
The computer2 is thinking..................
No. of possible moves from here: 4
<class '__main__.treeNode'>

Current configuration of the board: 



_______________________________________


|    _    ||    _    ||    O    |

|    _    ||    X    ||    O    |

|    X    ||    X    ||    O    |

_______________________________________
Player 2 has won!!
Current state of the board:------


_______________________________________


|    _    ||    _    ||    _    |

|    _    ||    _    ||    _    |

|    _    ||    _    ||    _    |

_______________________________________
Monte Carlo Agent Created
The computer1 is thinking..................
No. of possible moves from here: 9
<

...

No. of possible moves from here: 9
<class '__main__.treeNode'>

Current configuration of the board: 



_______________________________________


|    _    ||    _    ||    _    |

|    _    ||    X    ||    _    |

|    _    ||    _    ||    _    |

_______________________________________
Monte Carlo Agent Created
The computer2 is thinking..................
No. of possible moves from here: 8
<class '__main__.treeNode'>

Current configuration of the board: 



_______________________________________


|    _    ||    _    ||    _    |

|    _    ||    X    ||    _    |

|    _    ||    _    ||    O    |

_______________________________________
Monte Carlo Agent Created
The computer1 is thinking..................
No. of possible moves from here: 7
<class '__main__.treeNode'>

Current configuration of the board: 



_______________________________________


|    _    ||    _    ||    _    |

|    X    ||    X    ||    _    |

|    _    ||    _    ||    O    |

___________________________

No. of possible moves from here: 9
<class '__main__.treeNode'>

Current configuration of the board: 



_______________________________________


|    _    ||    _    ||    _    |

|    X    ||    _    ||    _    |

|    _    ||    _    ||    _    |

_______________________________________
Monte Carlo Agent Created
The computer2 is thinking..................
No. of possible moves from here: 8
<class '__main__.treeNode'>

Current configuration of the board: 



_______________________________________


|    _    ||    _    ||    _    |

|    X    ||    _    ||    O    |

|    _    ||    _    ||    _    |

_______________________________________
Monte Carlo Agent Created
The computer1 is thinking..................
No. of possible moves from here: 7
<class '__main__.treeNode'>

Current configuration of the board: 



_______________________________________


|    X    ||    _    ||    _    |

|    X    ||    _    ||    O    |

|    _    ||    _    ||    _    |

___________________________

No. of possible moves from here: 9
<class '__main__.treeNode'>

Current configuration of the board: 



_______________________________________


|    _    ||    _    ||    _    |

|    _    ||    X    ||    _    |

|    _    ||    _    ||    _    |

_______________________________________
Monte Carlo Agent Created
The computer2 is thinking..................
No. of possible moves from here: 8
<class '__main__.treeNode'>

Current configuration of the board: 



_______________________________________


|    _    ||    _    ||    _    |

|    O    ||    X    ||    _    |

|    _    ||    _    ||    _    |

_______________________________________
Monte Carlo Agent Created
The computer1 is thinking..................
No. of possible moves from here: 7
<class '__main__.treeNode'>

Current configuration of the board: 



_______________________________________


|    _    ||    _    ||    _    |

|    O    ||    X    ||    _    |

|    _    ||    _    ||    X    |

___________________________

No. of possible moves from here: 7
<class '__main__.treeNode'>

Current configuration of the board: 



_______________________________________


|    _    ||    X    ||    _    |

|    _    ||    X    ||    _    |

|    _    ||    _    ||    O    |

_______________________________________
Monte Carlo Agent Created
The computer2 is thinking..................
No. of possible moves from here: 6
<class '__main__.treeNode'>

Current configuration of the board: 



_______________________________________


|    _    ||    X    ||    _    |

|    O    ||    X    ||    _    |

|    _    ||    _    ||    O    |

_______________________________________
Monte Carlo Agent Created
The computer1 is thinking..................
No. of possible moves from here: 5
<class '__main__.treeNode'>

Current configuration of the board: 



_______________________________________


|    _    ||    X    ||    _    |

|    O    ||    X    ||    _    |

|    _    ||    X    ||    O    |

___________________________

No. of possible moves from here: 9
<class '__main__.treeNode'>

Current configuration of the board: 



_______________________________________


|    _    ||    _    ||    _    |

|    _    ||    _    ||    _    |

|    X    ||    _    ||    _    |

_______________________________________
Monte Carlo Agent Created
The computer2 is thinking..................
No. of possible moves from here: 8
<class '__main__.treeNode'>

Current configuration of the board: 



_______________________________________


|    O    ||    _    ||    _    |

|    _    ||    _    ||    _    |

|    X    ||    _    ||    _    |

_______________________________________
Monte Carlo Agent Created
The computer1 is thinking..................
No. of possible moves from here: 7
<class '__main__.treeNode'>

Current configuration of the board: 



_______________________________________


|    O    ||    _    ||    _    |

|    _    ||    _    ||    X    |

|    X    ||    _    ||    _    |

___________________________

Results after 1000 games:
Player1 won 748 games, player 2 won 240 games, and 12 games eneded in a draw.